In [95]:
import pandas as pd

In [96]:
data = pd.read_excel('WholePremise_LoadShapeDatav11.xlsx', skiprows=1)
data.head

<bound method NDFrame.head of               City,State Springfield, IL Springfield, IL.1 Springfield, IL.2  \
0                 Sector      Commercial        Commercial        Commercial   
1           Heating Type        Electric          Electric          Electric   
2              Bldg Type   Office, Large     Office, Large     Office, Large   
3                 Region         Midwest           Midwest           Midwest   
4                    Day             HR1               HR2               HR3   
..                   ...             ...               ...               ...   
740  2001-12-28 00:00:00         3.40277           3.56706           3.65192   
741  2001-12-29 00:00:00         3.39528           3.46493           3.52094   
742  2001-12-30 00:00:00         3.86993           4.18424           4.31692   
743  2001-12-31 00:00:00         3.51558            3.5604           3.89187   
744                  NaN               1                 2                 3   

    Sprin

In [97]:
# data = data.filter(like='SAN ANTONIO, TX')

In [98]:
data.head

<bound method NDFrame.head of               City,State Springfield, IL Springfield, IL.1 Springfield, IL.2  \
0                 Sector      Commercial        Commercial        Commercial   
1           Heating Type        Electric          Electric          Electric   
2              Bldg Type   Office, Large     Office, Large     Office, Large   
3                 Region         Midwest           Midwest           Midwest   
4                    Day             HR1               HR2               HR3   
..                   ...             ...               ...               ...   
740  2001-12-28 00:00:00         3.40277           3.56706           3.65192   
741  2001-12-29 00:00:00         3.39528           3.46493           3.52094   
742  2001-12-30 00:00:00         3.86993           4.18424           4.31692   
743  2001-12-31 00:00:00         3.51558            3.5604           3.89187   
744                  NaN               1                 2                 3   

    Sprin

In [99]:
print(data.columns)

Index(['City,State', 'Springfield, IL', 'Springfield, IL.1',
       'Springfield, IL.2', 'Springfield, IL.3', 'Springfield, IL.4',
       'Springfield, IL.5', 'Springfield, IL.6', 'Springfield, IL.7',
       'Springfield, IL.8',
       ...
       'TUCSON, AZ.422', 'TUCSON, AZ.423', 'TUCSON, AZ.424', 'TUCSON, AZ.425',
       'TUCSON, AZ.426', 'TUCSON, AZ.427', 'TUCSON, AZ.428', 'TUCSON, AZ.429',
       'TUCSON, AZ.430', 'TUCSON, AZ.431'],
      dtype='object', length=16369)


In [100]:
#See your choices for locations
def peek_locations(data):
    locations = set(data.columns.values)
    print(locations)
    return locations
    

In [101]:
#Create new DF with like locatios
def filter_location(data, location):
    data = data.filter(like=location)
    return data
    

In [102]:
#peek at your buildings
def peek_buildings(data):
    print(set(data.iloc[2].values))
    return list(set(data.iloc[2].values))


In [103]:
#Input a list of buildings you want to build loadshapes for
def make_buildings_df(data, buildings='All'):
    if buildings =='All':
        buildings = list(set(data.iloc[2].values))
    print(buildings)
    df_dict = {}
    for i in buildings:
        buildings = data.columns[data.iloc[2] == i]
        df = data[buildings]
        #only support electric for now, assume military bases are rural enough that they cannot support nat gas.
        source = df.columns[df.iloc[1]=='Electric']
        df = df[source].dropna()
        df_dict[i] = df
#     print(df_dict)
    return df_dict


In [104]:
#Driver
locations = peek_locations(data)
data = filter_location(data, 'SAN ANTONIO, TX')
buildings = get_buildings(data)
df = make_buildings_df(data)

{'AUSTIN, TX.378', 'HOUSTON, TX.321', 'TOLEDO, OH.130', 'LAS VEGAS, NV.263', 'BROWNSVILLE, TX.68', 'FRESNO, CA.204', 'HOUSTON, TX.137', 'GREENSBORO, NC.167', 'GREENSBORO, NC.389', 'FORT WORTH, TX.317', 'BOSTON, MA.332', 'GREENSBORO, NC.225', 'NEWARK, NJ.79', 'FORT WORTH, TX.159', 'AUSTIN, TX.251', 'Miami,FL.163', 'Toledo, OH.125', 'Miami,FL.185', 'ROCHESTER, NY.381', 'AUSTIN, TX.145', 'ROCHESTER, NY.131', 'PITTSBURGH, PA.326', 'CHICAGO, IL.284', 'HELENA, MT.281', 'Springfield, IL.16', 'Springfield, IL.70', 'Akron, OH.192', 'MEDFORD, OR.248', 'FRESNO, CA.339', 'CHICAGO, IL.83', 'CHARLOTTE, NC.413', 'DAYTON, OH.24', 'COLUMBUS, OH.171', 'LAS VEGAS, NV.13', 'BURLINGTON, VT.344', 'TOLEDO, OH.82', 'GREENSBORO, NC.91', 'PITTSBURGH, PA.251', 'ATLANTA, GA.344', 'Lansing, MI.151', 'AKRON, OH.78', 'Akron, OH.168', 'FRESNO, CA.21', 'LAS VEGAS, NV.206', 'PHOENIX, AZ.20', 'BOSTON, MA.116', 'EL PASO, TX.101', 'BROWNSVILLE, TX.4', 'ATLANTIC CITY, NJ.73', 'Dayton, OH.2', 'LAS VEGAS, NV.254', 'GREENSBOR

In [105]:
print(df)

{'Hotel':     SAN ANTONIO, TX.72 SAN ANTONIO, TX.73 SAN ANTONIO, TX.74  \
0           Commercial         Commercial         Commercial   
1             Electric           Electric           Electric   
2                Hotel              Hotel              Hotel   
3                South              South              South   
4                  HR1                HR2                HR3   
..                 ...                ...                ...   
365            1.96729             1.9199            1.91298   
366            1.98624            2.05109            2.04564   
367            3.39848            3.81913            3.77431   
368            4.29593            4.31885            4.26593   
369            1.66444           0.927676            0.96611   

    SAN ANTONIO, TX.75 SAN ANTONIO, TX.76 SAN ANTONIO, TX.77  \
0           Commercial         Commercial         Commercial   
1             Electric           Electric           Electric   
2                Hotel       

In [118]:
load_shape_arr = [0]*8760
for building in df.values():
    idx = 0
    rows = len(building)
    cols = len(building.columns)

    for row in range(5,(rows)):
        for col in range((cols)):
            load_shape_arr[idx] = building.iloc[row,col]
            idx+=1
            
print(load_shape_arr)
print(len(load_shape_arr))


[1.32358713617261, 1.32652612949525, 1.26022489224465, 1.19059916194556, 1.17352046308266, 1.17651132792805, 1.17065809814588, 1.19633921749629, 1.33534664615793, 1.5479467977102, 1.66828047818462, 1.67631231192196, 1.65521592784302, 1.634085355715, 1.64832173091128, 1.63152007313575, 1.5296502969611, 1.45230985655234, 1.41873483455915, 1.39041887763308, 1.39981705443538, 1.43322231508142, 1.43905314579695, 1.37815694601358, 1.28002622806568, 1.21990347680142, 1.19695458791883, 1.18953324175808, 1.19557048605832, 1.21196369248714, 1.26816142825938, 1.60729998655736, 2.26663650925405, 2.73609665833732, 2.80173988923395, 2.68723066213139, 2.54932451317406, 2.44897632822817, 2.38999438755733, 2.35213880164293, 2.27509793195738, 2.11077543893627, 1.94841309016848, 1.85904491327672, 1.74441247710682, 1.55213592685666, 1.38360352401474, 1.3020580861007, 1.2689091790738, 1.24290151942221, 1.21942037634039, 1.21182690669783, 1.21800417296807, 1.23477758747604, 1.29227871885081, 1.6392829268846

In [43]:
template = pd.read_csv('load_profile_template.csv')
print(template)

      Hour  Load (kW)
0        1        NaN
1        2        NaN
2        3        NaN
3        4        NaN
4        5        NaN
...    ...        ...
8755  8756        NaN
8756  8757        NaN
8757  8758        NaN
8758  8759        NaN
8759  8760        NaN

[8760 rows x 2 columns]
